In [6]:
import numpy as np
import pandas as pd
import umap
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
#from .autonotebook import tqdm as notebook_tqdm

data = pd.read_csv(r'C:\Users\jdorv\Coding Fun\Bank Innovation\wrds_bank_data_MERGED.csv')


In [ ]:
def calculate_ratios(df):
    """Calculate ratios using ONLY universally reported fields"""
    print(f"\n{'='*80}")
    print("CALCULATING FINANCIAL RATIOS (UNIVERSAL FIELDS ONLY)")
    print(f"{'='*80}")
    
    ratios = df.copy()
    
    def safe_divide(numerator, denominator):
        result = numerator / denominator
        result = result.replace([np.inf, -np.inf], np.nan)
        return result
    
    # === CORE INNOVATION METRICS (3 ratios) ===
    ratios['tech_investment_ratio'] = safe_divide(
        df['riad4092'], df['rcfd2_rcfd2170']
    ) * 1000
    print("✓ Tech Investment Ratio")
    
    ratios['nib_deposit_ratio'] = safe_divide(
        df['rcon2_rcon6631'], df['rcon2_rcon2200']
    ) * 100
    print("✓ NIB Deposit Ratio (Digital Banking Proxy)")
    
    ratios['service_charge_intensity'] = safe_divide(
        df['riad4080'], df['rcon2_rcon2200']
    ) * 1000
    print("✓ Service Charge Intensity")
    
    # === EFFICIENCY METRICS (2 ratios) ===
    ratios['efficiency_ratio'] = safe_divide(
        df['riad4093'], df['riad4074'] + df['riad4079']
    ) * 100
    print("✓ Efficiency Ratio")
    
    ratios['nonint_income_pct'] = safe_divide(
        df['riad4079'], df['riad4074'] + df['riad4079']
    ) * 100
    print("✓ Noninterest Income %")
    
    # === BALANCE SHEET (3 ratios) ===
    ratios['loans_to_assets'] = safe_divide(
        df['rcfd2_rcfd2122'], df['rcfd2_rcfd2170']
    ) * 100
    print("✓ Loans-to-Assets")
    
    ratios['equity_to_assets'] = safe_divide(
        df['rcfd2_rcfd3210'], df['rcfd2_rcfd2170']
    ) * 100
    print("✓ Equity-to-Assets")
    
    ratios['deposits_to_assets'] = safe_divide(
        df['rcon2_rcon2200'], df['rcfd2_rcfd2170']
    ) * 100
    print("✓ Deposits-to-Assets")
    
    # === PROFITABILITY (2 ratios) ===
    ratios['roa'] = safe_divide(
        df['riad4340'], df['rcfd2_rcfd2170']
    ) * 100
    print("✓ ROA")
    
    ratios['roe'] = safe_divide(
        df['riad4340'], df['rcfd2_rcfd3210']
    ) * 100
    print("✓ ROE")
    
    # === DEPOSIT MIX (1 ratio) ===
    ratios['nontrans_deposits_pct'] = safe_divide(
        df['rcon2_rcon2215'], df['rcon2_rcon2200']
    ) * 100
    print("✓ Nontransaction Deposits %")
    
    print(f"\n✓ Created {11} ratios using universally reported fields")
    return ratios
def handle_split_codes(df):
    """
    Handle RCFD/RCON split that occurred around 2011.
    Strategy: Use RCFD (consolidated) as primary, fill with RCON if needed
    """
    print(f"\n{'='*80}")
    print("HANDLING SPLIT TIME CODES (2011 Transition)")
    print(f"{'='*80}")
    
    # Define split code pairs (RCFD consolidated, RCON domestic)
    split_pairs = [
        ('rcfd2170', 'rcon2170'),  # Total Assets
        ('rcfd2122', 'rcon2122'),  # Total Loans
        ('rcfd3210', 'rcon3210'),  # Total Equity
        ('rcfd3123', 'rcon3123'),  # Allowance
    ]
    
    for rcfd_col, rcon_col in split_pairs:
        if rcfd_col in df.columns and rcon_col in df.columns:
            # Fill missing RCFD with RCON (for consistency)
            original_na = df[rcfd_col].isna().sum()
            df[rcfd_col] = df[rcfd_col].fillna(df[rcon_col])
            filled = original_na - df[rcfd_col].isna().sum()
            
            if filled > 0:
                print(f"✓ {rcfd_col}: Filled {filled:,} missing values with {rcon_col}")
            
            # Drop the RCON duplicate to avoid redundancy
            df = df.drop(columns=[rcon_col])
    
    return df
def prepare_clustering_features(df):
    """Select universal coverage features AND keep identifiers"""
    print(f"\n{'='*80}")
    print("PREPARING CLUSTERING FEATURES - UNIVERSAL COVERAGE ONLY")
    print(f"{'='*80}")
    
    # RIAD columns - Universal coverage (<2% missing)
    riad_universal = [
        'riad4010', 'riad4012', 'riad4020', 'riad4073', 'riad4074', 'riad4079',
        'riad4080', 'riad4092', 'riad4093', 'riad4107', 'riad4115', 'riad4135',
        'riad4150', 'riad4180', 'riad4217', 'riad4230', 'riad4266', 'riad4267',
        'riad4300', 'riad4301', 'riad4302', 'riad4313', 'riad4340', 'riad4356',
        'riad4415', 'riad4435', 'riad4436', 'riad4460', 'riad4470', 'riad4498',
        'riad4499', 'riad4507', 'riad4508', 'riad4518', 'riad4605', 'riad4608',
        'riad4628', 'riad4635', 'riad4638', 'riad4644', 'riad4769'
    ]
    
    # RCON columns - Universal coverage (<2% missing)
    rcon_universal = [
        'rcon1_rcon1590', 'rcon1_rcon1754', 'rcon1_rcon1766', 'rcon1_rcon1773',
        'rcon1_rcon3000', 'rcon1_rcon3123', 'rcon2_rcon0081', 'rcon2_rcon1420',
        'rcon2_rcon1460', 'rcon2_rcon2122', 'rcon2_rcon2150', 'rcon2_rcon2170',
        'rcon2_rcon2200', 'rcon2_rcon2202', 'rcon2_rcon2215', 'rcon2_rcon3210',
        'rcon2_rcon6631'
    ]
    
    # Ratios - Successfully calculated (<2% missing)
    ratios_clean = [
        'nib_deposit_ratio',
        'service_charge_intensity',
        'efficiency_ratio',
        'nonint_income_pct',
        'nontrans_deposits_pct'
    ]
    
    # Identifiers - MUST KEEP THESE
    identifiers = ['rssd9001', 'rssd9999', 'rssd9017', 'year', 'quarter']
    
    # Check which identifiers exist
    existing_identifiers = [col for col in identifiers if col in df.columns]
    if len(existing_identifiers) < len(identifiers):
        missing = [col for col in identifiers if col not in df.columns]
        print(f"\n⚠️  WARNING: Missing identifier columns: {missing}")
        print(f"   Available columns for identifiers: {[col for col in df.columns if 'rssd' in col.lower() or 'year' in col.lower() or 'quarter' in col.lower()]}")
    
    # Combine all feature columns
    feature_cols = riad_universal + rcon_universal + ratios_clean
    
    # Check which features actually exist
    existing_features = [col for col in feature_cols if col in df.columns]
    missing_features = [col for col in feature_cols if col not in df.columns]
    
    print(f"\n📊 Feature selection:")
    print(f"  Expected: {len(feature_cols)} features")
    print(f"  Found: {len(existing_features)} features")
    if missing_features:
        print(f"  Missing: {len(missing_features)} features")
    
    # Create dataset with BOTH identifiers AND features
    all_cols = existing_identifiers + existing_features
    df_subset = df[all_cols].copy()
    
    print(f"\n🔍 Missing value summary:")
    total_missing = df_subset.isna().sum().sum()
    total_cells = df_subset.shape[0] * df_subset.shape[1]
    missing_pct = (total_missing / total_cells) * 100
    print(f"  Total missing values: {total_missing:,} ({missing_pct:.2f}% of all cells)")
    
    # Drop rows with ANY missing values
    print(f"\n{'='*80}")
    print("REMOVING ROWS WITH MISSING VALUES")
    print(f"{'='*80}")
    
    before_rows = len(df_subset)
    df_clean = df_subset.dropna()
    after_rows = len(df_clean)
    
    print(f"  Before: {before_rows:,} rows")
    print(f"  After:  {after_rows:,} rows")
    print(f"  Retained: {after_rows/before_rows*100:.1f}%")
    
    print(f"\n{'='*80}")
    print(f"✓ FINAL DATASET READY FOR CLUSTERING")
    print(f"{'='*80}")
    print(f"  Observations: {len(df_clean):,}")
    print(f"  Identifiers: {len(existing_identifiers)} columns - {existing_identifiers}")
    print(f"  Features: {len(existing_features)} columns")
    print(f"  Total columns: {len(df_clean.columns)}")
    print(f"  Missing values: {df_clean.isna().sum().sum()}")
    print(f"\n  Ready for: Z-score standardization → UMAP → HDBSCAN")
    
    # Return COMPLETE dataframe (identifiers + features)
    return df_clean, existing_features



In [ ]:
# Then your workflow stays the same:
df = data.copy()
df['report_date'] = pd.to_datetime(df['rssd9999'], format='%Y%m%d', errors='coerce')
df['year'] = df['report_date'].dt.year
df['quarter'] = df['report_date'].dt.quarter

df = handle_split_codes(df)
df_ratios = calculate_ratios(df)
df_umap, feature_names = prepare_clustering_features(df_ratios)

print(f"\n✓ Ready to cluster with {df_umap.shape[0]:,} observations and {df_umap.shape[1]} features")


HANDLING SPLIT TIME CODES (2011 Transition)

CALCULATING FINANCIAL RATIOS (UNIVERSAL FIELDS ONLY)
✓ Tech Investment Ratio
✓ NIB Deposit Ratio (Digital Banking Proxy)
✓ Service Charge Intensity
✓ Efficiency Ratio
✓ Noninterest Income %
✓ Loans-to-Assets
✓ Equity-to-Assets
✓ Deposits-to-Assets
✓ ROA
✓ ROE
✓ Nontransaction Deposits %

✓ Created 11 ratios using universally reported fields

PREPARING CLUSTERING FEATURES - UNIVERSAL COVERAGE ONLY

📊 Feature selection:
  Expected: 63 features
  Found: 63 features

🔍 Missing value summary:
  Total missing values: 52,576 (0.27% of all cells)

REMOVING ROWS WITH MISSING VALUES
  Before: 296,463 rows
  After:  289,498 rows
  Retained: 97.7%

✓ FINAL DATASET READY FOR CLUSTERING
  Observations: 289,498
  Identifiers: 3 columns - ['rssd9001', 'rssd9017', 'rssd9999']
  Features: 63 columns
  Total columns: 66
  Missing values: 0

  Ready for: Z-score standardization → UMAP → HDBSCAN

✓ Ready to cluster with 289,498 observations and 66 features


In [28]:
print(df_umap.columns.tolist)

<bound method IndexOpsMixin.tolist of Index(['rssd9001', 'riad4010', 'riad4012', 'riad4020', 'riad4073', 'riad4074',
       'riad4079', 'riad4080', 'riad4092', 'riad4093', 'riad4107', 'riad4115',
       'riad4135', 'riad4150', 'riad4180', 'riad4217', 'riad4230', 'riad4266',
       'riad4267', 'riad4300', 'riad4301', 'riad4302', 'riad4313', 'riad4340',
       'riad4356', 'riad4415', 'riad4435', 'riad4436', 'riad4460', 'riad4470',
       'riad4498', 'riad4499', 'riad4507', 'riad4508', 'riad4518', 'riad4605',
       'riad4608', 'riad4628', 'riad4635', 'riad4638', 'riad4644', 'riad4769',
       'rcon1_rcon1590', 'rcon1_rcon1754', 'rcon1_rcon1766', 'rcon1_rcon1773',
       'rcon1_rcon3000', 'rcon1_rcon3123', 'rcon2_rcon0081', 'rcon2_rcon1420',
       'rcon2_rcon1460', 'rcon2_rcon2122', 'rcon2_rcon2150', 'rcon2_rcon2170',
       'rcon2_rcon2200', 'rcon2_rcon2202', 'rcon2_rcon2215', 'rcon2_rcon3210',
       'rcon2_rcon6631', 'nib_deposit_ratio', 'service_charge_intensity',
       'efficiency_